In [1]:
import numpy as np
import pandas as pd
import unicodedata
import re
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


In [2]:
data = pd.read_csv('output.csv')


In [3]:
# 1. Loại bỏ khoảng trắng và điền NaN
data['attributes'] = data['attributes'].str.strip()
data['attributes'] = data['attributes'].fillna('Không rõ')  # Loại bỏ inplace=True

# 2. Mã hóa các thuộc tính
attributes_dummies = data['attributes'].str.get_dummies(sep=', ')

# 3. Ghép các cột đã mã hóa vào DataFrame gốc
data = pd.concat([data, attributes_dummies], axis=1)

# 4. Kiểm tra kết quả
print(data.head())



   province  district                   street  price_unit  price_value  \
0         1         5  Mặt phố C1 TT Nghĩa Tân           1         5.50   
1         1         5                  Yên Hoà           1         8.58   
2         1         5                 Cầu Giấy           1         8.65   
3         1         5              Hồ Tùng Mậu           1        28.50   
4         1         5                 Cầu Giấy           1         8.90   

   price_absolute   area  type  front  room  toilet  floor  \
0          5500.0   70.0     1    7.0     2       2      1   
1          8580.0   45.0     1    5.0     5       4      4   
2          8650.0   43.0     1    3.6     6       1      5   
3         28500.0  100.0     1    6.9    29       0      8   
4          8900.0   35.0     1    3.4     6       2      5   

                    attributes  Chung cư  Dòng tiền  Không rõ  Kinh doanh  \
0  Ô tô, Kinh doanh, Dòng tiền         0          1         0           1   
1                     

In [4]:
 # Hàm chuẩn hóa tên đường
def normalize_street_name(street_name):
    # Loại bỏ dấu tiếng Việt
    street_name = unicodedata.normalize('NFKD', street_name).encode('ascii', 'ignore').decode('utf-8')
    # Chuyển thành chữ thường
    street_name = street_name.lower()
    # Loại bỏ các ký tự đặc biệt và khoảng trắng thừa
    street_name = re.sub(r'[^a-z0-9\s]', '', street_name)
    street_name = re.sub(r'\s+', ' ', street_name).strip()
    return street_name

# Áp dụng hàm chuẩn hóa lên cột 'street'
data['street_normalized'] = data['street'].apply(normalize_street_name)

# Thực hiện one-hot encoding sau khi chuẩn hóa
street_encoded = pd.get_dummies(data['street_normalized'], prefix='street')

# Nối cột mới với DataFrame gốc
data = pd.concat([data, street_encoded], axis=1)

# Xóa cột 'street_normalized' nếu không cần thiết nữa
data.drop(columns=['street_normalized'], inplace=True)

# Kiểm tra lại dữ liệu sau khi xử lý
print(data.head())

   province  district                   street  price_unit  price_value  \
0         1         5  Mặt phố C1 TT Nghĩa Tân           1         5.50   
1         1         5                  Yên Hoà           1         8.58   
2         1         5                 Cầu Giấy           1         8.65   
3         1         5              Hồ Tùng Mậu           1        28.50   
4         1         5                 Cầu Giấy           1         8.90   

   price_absolute   area  type  front  room  ...  street_truong cong giai  \
0          5500.0   70.0     1    7.0     2  ...                    False   
1          8580.0   45.0     1    5.0     5  ...                    False   
2          8650.0   43.0     1    3.6     6  ...                    False   
3         28500.0  100.0     1    6.9    29  ...                    False   
4          8900.0   35.0     1    3.4     6  ...                    False   

   street_tt van cong mai dich street_tt van hoa nghe thuat mai dich  \
0             

In [5]:
data = data.drop(columns=['street'])

data = data.drop(columns=['attributes'])

data = data.drop(columns=['price_value'])

X = data.drop('price_absolute', axis=1)
y = data['price_absolute']

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [11]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [13]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
import numpy as np


In [15]:
# Tăng cường param_grid với nhiều giá trị hơn cho các tham số
param_grid_lgb = {
    'n_estimators': [100, 200, 300, 400, 500, 600],  # Số lượng cây tăng thêm
    'max_depth': [-1, 3, 4, 5, 6, 7, 8, 10],  # Độ sâu của cây
    'learning_rate': [0.005, 0.01, 0.02, 0.05, 0.1, 0.15],  # Tỷ lệ học thêm một số giá trị nhỏ hơn
    'num_leaves': [20, 31, 40, 50, 60],  # Số lượng lá tối đa
    'min_child_samples': [5, 10, 20, 30, 40, 50],  # Thêm các giá trị nhỏ hơn cho số lượng mẫu tối thiểu trong mỗi lá
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Tỷ lệ mẫu cho mỗi cây
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Tỷ lệ cột
    'reg_alpha': [0, 0.01, 0.1, 1, 5],  # Điều chỉnh độ phức tạp
    'reg_lambda': [0, 0.01, 0.1, 1, 5],  # Điều chỉnh độ phức tạp
}

In [16]:
# Khởi tạo mô hình LightGBM
lgb_model = lgb.LGBMRegressor(objective='regression', random_state=101)

In [17]:
# Thiết lập RandomizedSearchCV với Early Stopping
random_search_lgb = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_grid_lgb,
    n_iter=200,  # Tăng số lần thử nghiệm
    scoring='neg_mean_squared_error',  # Sử dụng MSE làm metric
    cv=7,  # Tăng số lượng cross-validation folds
    verbose=1,
    n_jobs=-1,
    random_state=101
)

In [22]:
# Thêm các tham số bổ sung vào fit_params
fit_params = {
    'eval_set': [(X_train, y_train), (X_test, y_test)],
    'eval_metric': 'rmse',
    'early_stopping_rounds': 50,
    'verbose': 0  # Không hiển thị thông tin huấn luyện
}


In [23]:
# Huấn luyện RandomizedSearchCV với fit_params
random_search_lgb.fit(X_train, y_train, **fit_params)

Fitting 7 folds for each of 200 candidates, totalling 1400 fits


ValueError: 
All the 1400 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1400 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Huy\PycharmProjects\pythonProject25\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
TypeError: LGBMRegressor.fit() got an unexpected keyword argument 'early_stopping_rounds'
